In [285]:
# imports
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from tqdm import tqdm
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch.nn.utils.rnn as RNN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

In [286]:
data = pd.read_csv('data.csv', delimiter=';')


data

,RECORDING_SESSION_LABEL,trial,IA_ID,item,list,IA_LABEL,wordlength,condition,is_critical,is_spill1,...,duration_firstpass,duration_firstfixation,fix_count,avg_pupil,IA_REGRESSION_IN_COUNT,IA_REGRESSION_OUT_COUNT,saccade_length,saccade_duration,go_past_time,sentenceCondition
0,10m23r2,12,1,1,2,Viel,4,none,0,0,...,236,236,1,1408.000000,0,0,72.376792,16,236,control
1,10m23r2,12,2,1,2,Geld,4,none,0,0,...,424,264,3,1379.333333,2,0,71.519648,160,424,control
2,10m23r2,12,3,1,2,wurde,5,none,0,0,...,0,0,0,0.000000,0,1,0.000000,0,0,control
3,10m23r2,12,4,1,2,"investiert,",11,none,0,0,...,420,268,3,1290.000000,1,0,65.401223,12,420,control
4,10m23r2,12,5,1,2,bevor,5,none,0,0,...,296,296,2,1242.500000,1,0,42.311819,12,296,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11171,9m23r1,10,12,28,1,Verteidigung,12,filler,0,0,...,168,168,2,497.000000,0,0,91.810675,20,168,filler
11172,9m23r1,10,13,28,1,Europas,7,none,0,0,...,224,224,1,493.000000,0,0,119.788355,24,224,filler
11173,9m23r1,10,14,28,1,erhöht,6,none,0,0,...,332,332,1,472.000000,0,0,95.117033,20,332,filler
11174,9m23r1,10,15,28,1,werden,6,none,0,0,...,244,244,1,477.000000,0,0,58.649126,16,244,filler


In [287]:
data.keys()

Index(['RECORDING_SESSION_LABEL', 'trial', 'IA_ID', 'item', 'list', 'IA_LABEL',
       'wordlength', 'condition', 'is_critical', 'is_spill1', 'is_spill2',
       'is_spill3', 'filler', 'LF', 'HF', 'function_word', 'other_filler',
       'composite', 'fixation_duration', 'duration_firstpass',
       'duration_firstfixation', 'fix_count', 'avg_pupil',
       'IA_REGRESSION_IN_COUNT', 'IA_REGRESSION_OUT_COUNT', 'saccade_length',
       'saccade_duration', 'go_past_time', 'sentenceCondition'],
      dtype='object')

In [288]:
data = pd.read_csv('data.csv', delimiter=';')

dropped = data.copy()

dropped[["condition", "sentenceCondition"]] = dropped[["condition", "sentenceCondition"]].map(lambda x: x.replace("none", "0"))
dropped[["condition", "sentenceCondition"]] = dropped[["condition", "sentenceCondition"]].map(lambda x: x.replace("control", "0"))
dropped[["condition", "sentenceCondition"]] = dropped[["condition", "sentenceCondition"]].map(lambda x: x.replace("pseudo", "1"))
dropped[["condition", "sentenceCondition"]] = dropped[["condition", "sentenceCondition"]].map(lambda x: x.replace("filler", "0"))

dropped.drop(["condition", "IA_ID", "item", "list", "IA_LABEL", "wordlength", "is_critical", 
              'is_spill1', 'is_spill2', 'is_spill3', 'filler', 'LF', 'HF', 'function_word', 'other_filler', "composite"], axis=1, inplace=True)

sentences = dropped.groupby(['RECORDING_SESSION_LABEL', 'trial'])

In [289]:
label_array = np.array([])
features_array = []
for item in sentences:
    label_array = np.append(label_array, item[1]["sentenceCondition"].unique().astype(int).item())
    features = item[1].drop(['RECORDING_SESSION_LABEL', 'trial', 'sentenceCondition'], axis=1).to_numpy()
    #print(features.shape)
    features_array.append(features)

def pad_to_same_size(lists):
    maxlen = max([len(l) for l in lists])
    return [np.concatenate((np.zeros((maxlen - l.shape[0], l.shape[1])), l), axis=0) for l in lists]
lengths = np.array([len(l) for l in features_array])
padded_features_array = np.array(pad_to_same_size(features_array))
print(label_array.shape, padded_features_array.shape, lengths.shape)

(851,) (851, 18, 10) (851,)


In [290]:
print(1 - label_array.sum()/len(label_array))

0.900117508813161


In [291]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        features = self.features[index]
        label = self.labels[index]
        return features, label

    def __len__(self):
        return len(self.features)

dataset = CustomDataset(features=padded_features_array, labels=label_array)

In [292]:
print(len(dataset))

851


In [293]:
def split_data(dataset, batch_size):
    
    generator = torch.Generator().manual_seed(42)
    train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1], generator=generator)
    #train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, validation_dataloader, test_dataloader

In [294]:
batch_size = 32

train_dataloader, validation_dataloader, test_dataloader = split_data(dataset, batch_size)

In [295]:
print(len(train_dataloader), len(validation_dataloader), len(test_dataloader))
for batch in train_dataloader:
    input, target = batch
    print(input.shape, target.shape)

22 3 3
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([32, 18, 10]) torch.Size([32])
torch.Size([9, 18, 10]) torch.Size([9])


In [296]:
def train_test(model, dataloader, optimizer, training="train"):
   
    loss_function = torch.nn.CrossEntropyLoss()

    if training == "train":
        model.train()
    elif training == "validation":
        model.eval()
    elif training == "test":
        model.eval()
    else:
        raise ValueError("training argument must be either 'train', 'validation' or 'test'")
        
    total = 0
    correct = 0
    cumulative_loss = 0
    prediction_list = []
    label_list = []
    for sample in dataloader:
        input, targets = sample[0].float().to(device), sample[1].type(torch.LongTensor).to(device)
        output = model(input)
        loss_value = loss_function(output, targets)
        cumulative_loss += loss_value.item()

        if training == "train":
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
        predictions = output.to('cpu').detach().numpy().argmax(axis=1)
        target_labels = targets.to('cpu').detach().numpy()
        total += len(predictions)
        correct += accuracy_score(target_labels, predictions, normalize=False)
        prediction_list.extend(predictions)
        label_list.extend(target_labels)  
    f1 = f1_score(label_list, prediction_list)
    accuracy = accuracy_score(label_list, prediction_list)
    confusion = confusion_matrix(label_list, prediction_list)

    return cumulative_loss, accuracy, f1, confusion

In [297]:
class TuneableModel(torch.nn.Module):
    def __init__(self, input_size, layer_size, dropout_rate, n_layers):
        super(TuneableModel, self).__init__()
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=layer_size, bidirectional=False, 
                                  num_layers=n_layers, batch_first=True, dropout=dropout_rate)
        self.output_layer = torch.nn.Linear(layer_size, 2)
        self.batchnorm = torch.nn.BatchNorm1d(layer_size)
        self.activation = torch.nn.ReLU()
        self.linear = torch.nn.Linear(layer_size, layer_size)

    def forward(self, x):

        x = self.lstm(x)
        x = x[0][:, -1, :]
        x = self.batchnorm(x)
        x = self.activation(x)
        x = self.linear(x)
        x = self.activation(x)
        x = self.output_layer(x)
        return x

In [302]:
# Training sample
def evaluate(params):
    dropout, hidden_size, learning_rate, batch_size, n_hidden = params

    max_epochs = 1000
    max_patience = 10
    
    PATH = "model.pt"

    train_dataloader, validation_dataloader, test_dataloader = split_data(dataset, batch_size)
    last_loss = 1000000
    torch.manual_seed(seed)
    input_size = train_dataloader.dataset[0][0].shape[1]
    model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(max_epochs):
        # training
        train_loss, train_accuracy, train_f1, train_confusion = train_test(model, train_dataloader, optimizer, training="train")
        train_loss, train_accuracy, train_f1 = round(train_loss, 2), round(train_accuracy, 4), round(train_f1, 2)
        # validation at end of epoch
        validation_loss, validation_accuracy, validation_f1, validation_confusion = train_test(model, validation_dataloader, optimizer, training="validation")
        validation_loss, validation_accuracy, validation_f1 = round(validation_loss, 2), round(validation_accuracy, 4), round(validation_f1, 2)
        if validation_loss < last_loss:
            last_loss = validation_loss
            current_patience = 0
        else:
            if current_patience == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': last_loss,
                    }, PATH)
            current_patience += 1
        if current_patience == max_patience:
            #print(f"Early stopping at epoch {epoch}")
            break   
        #if epoch % 10 == 0 and epoch != 0:
            #print(f"Epoch {epoch}\nvalidation loss: {round(validation_loss, 2)}\nvalidation accuracy: {validation_accuracy*100}%\nvalidation f1: {validation_f1}\n")

    # Testing once patience is reached
    torch.manual_seed(seed)
    model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    test_loss, test_accuracy, test_f1, test_confusion = train_test(model, test_dataloader, optimizer, training="test")
    test_loss, test_accuracy, test_f1 = round(test_loss, 2), round(test_accuracy, 4), round(test_f1, 2)
    #print(f"Model {i} at epoch {checkpoint['epoch']} test results: accuracy: {test_accuracy*100}% f1: {test_f1}")

    return test_accuracy, test_f1, test_confusion
    # print(f"Average accuracy: {round(np.mean(accuracies), 2)}%")
    # print(f"Average f1: {round(np.mean(f1s), 2)}")


In [299]:
params_nn ={
    #'dropout': [x/10 for x in list(range(0, 11, 3))][:3],
    'dropout': [0.5],
    'hidden_size': list(range(100, 101, 100)),
    'learning_rate':[0.01, 0.001, 0.0001, 0.00001],
    'batch_size':[16, 32],
    'n_hidden': list(range(2, 5, 1))
}
parameter_expansion = list(product(*params_nn.values()))
print(len(parameter_expansion))

24


In [300]:
results = {}
for i, p in tqdm(enumerate(parameter_expansion)):
    dropout, hidden_size, learning_rate, batch_size, n_hidden = p
    accuracy, f1 = evaluate(p)
    model_performance = {"dropout": dropout, "hidden_size": hidden_size, "learning_rate": learning_rate, 
              "batch_size": batch_size, "n_hidden": n_hidden, "accuracy": accuracy, "f1": f1}
    results[i] = model_performance
    print(model_performance)

0it [00:01, ?it/s]


ValueError: too many values to unpack (expected 3)

In [309]:
best_params = (0.5, 400, 0.0002, 32, 5)
accuracy, f1, confusion = evaluate(best_params)
print("Accuracy and f1 for best parameters: ", accuracy, f1)
print("Confusion matrix:\n", confusion)
print("Best parameters: ", best_params)

In [ ]:
results_dataframe = pd.DataFrame.from_dict(results)